# Giới thiệu chung về thuật toán
### CatBoost là một biến thể gradient boosting trên Decision Tree, được thiết kế để xử lý biến phân loại một cách “thông minh” và giảm overfitting / target leakage tốt hơn các thuật toán như XGBoost, LightGBM

#### Gradient boosting (GBDT) = xây nhiều cây nhỏ tuần tự:

- Cây đầu học dự đoán mục tiêu.

- Cây sau học sửa lỗi (residual / gradient) của tổ hợp các cây trước.

- Cuối cùng mô hình = tổng (hoặc tổ hợp) của tất cả cây.

#### Vấn đề của bản GBDT “truyền thống”:

- Dễ overfit.

- Xử lý biến phân loại phải one-hot / label encoding → mất thông tin hoặc gây nhiễu.

- Dễ bị target leakage / prediction shift khi dùng thống kê mục tiêu (mean target theo category) được tính từ cả tập train.

### Điểm cải tiến của CatBoost
#### Oblivious (symmetric) trees
- Ở mỗi tầng, tất cả node dùng cùng một điều kiện split.

- Cây sâu d luôn có 2^d lá; đường đi biểu diễn bằng dãy bit (trái/phải).
#### Xử lý biến phân loại bằng Target Statistics (TS)
- Với một category c, thay vì one-hot, ta dùng:

-   $TS(c)≈ \frac{∑y+a⋅Pn+a}{n + a}$

#### Ordered Boosting
- CatBoost trộn ngẫu nhiên các mẫu thành một permutation.

- Đối với mỗi mẫu tại vị trí i, TS được tính chỉ từ các mẫu đứng trước nó (1…i-1) trong permutation

- Khi xây cây thứ k, dự đoán và gradient cho một mẫu được tính từ mô hình dựa trên prefix trước đó trong permutation, thay vì toàn bộ
	​


## 1. Cài thư viện

In [12]:
# !pip install catboost

## 2. Khai báo mô hình
Sử dụng model CatBoostRegressor từ thư viện catboost

In [13]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [14]:
base_model = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    bootstrap_type="Bayesian",
    random_seed=42,
    verbose=False,      # tắt spam log khi CV
    od_type="Iter",
    od_wait=100         # early stopping trong từng lần train
)

## 3. Sử dụng RandomizedSearchCV để tìm bộ tham số

### 3.1 Bộ tham số dùng để tìm kiếm

In [15]:
# =========  KHOẢNG TÌM KIẾM THAM SỐ (PARAM DISTRIBUTIONS) =========
param_dist = {
    "depth": [4, 5, 6, 7, 8, 9, 10],
    "learning_rate": np.linspace(0.01, 0.15, 15),
    "l2_leaf_reg": np.logspace(0, 3, 20),      # 1 -> 1000
    "bagging_temperature": np.linspace(0, 5, 11),
    "border_count": [64, 128, 254],
    "iterations": [1000, 2000, 3000, 4000]     # kết hợp với early stopping
}


### 3.2 Khai RandomizedSearchCV

In [16]:
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=40,     # số tổ hợp random thử (tăng nếu muốn kỹ hơn)
    scoring="neg_root_mean_squared_error",
    cv=5,          # 5-fold CV
    n_jobs=-1,
    verbose=2,
    refit=True,
    random_state=42
)

### 3.3 Fit RandomizedSearchCV với training data

In [ ]:
# read train data
path = "../data/processed/train.csv"
df = pd.read_csv(path)

feature = [
    c for c in df.select_dtypes(include=['float64', 'int64']).columns
    if c not in ['life_expectancy', 'year']
]
X = df[feature]
y = df['life_expectancy']

# fit search model
random_search.fit(X, y)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  25.7s
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  27.8s
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  32.3s
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  32.7s
[CV] END bagging_temperature=2.5, border_count=254, depth=7, iterations=2000, l2_leaf_reg=54.55594781168517, learning_rate=0.03; total time=  33.0s
[CV] END bagging_temperature=2.5, border_count=254, depth=7, iterations=2000, l2_leaf_reg=54.55594781168517, learning_rate=0.03; total time=  35.6s
[CV] END bagging_temperature=2.5, border_count=64,

### 3.4 Bộ tham số tốt nhất 

In [1]:
print("Best params:", random_search.best_params_)
print("Best CV RMSE:", -random_search.best_score_)

NameError: name 'random_search' is not defined

## 4. Evaluation

In [ ]:
# read train data
path = "../data/processed/val.csv"
df = pd.read_csv(path)

feature = [
    c for c in df.select_dtypes(include=['float64', 'int64']).columns
    if c not in ['life_expectancy', 'year']
]
X_test = df[feature]
y_test = df['life_expectancy']



best_model = random_search.best_estimator_

y_pred_test = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(((y_test - y_pred_test) ** 2).mean())
r2 = r2_score(y_test, y_pred_test)

print(f"Test MAE : {mae:.3f}")
print(f"Test RMSE: {rmse:.3f}")
print(f"Test R²  : {r2:.3f}")

- Best params: {'learning_rate': np.float64(0.06999999999999999), 'l2_leaf_reg': np.float64(2.976351441631318), 'iterations': 2000, 'depth': 9, 'border_count': 64, 'bagging_temperature': np.float64(0.5)}
- Best CV RMSE: 1.9391243329337335


- Test MAE : 0.956
- Test RMSE: 1.386
- Test R²  : 0.973